### ANÁLISE DE SENTIMENTO EM REVIEWS DE TRÊS ESTRELAS: UMA PROPOSTA DE CLASSIFICAÇÃO MULTIRRÓTULO

##### Seção 7. Preparação dos Dados para os Modelos de Aprendizado de Máquina

#### Importação das Bibliotecas

In [ ]:
import pandas as pd
import re
import nltk
import numpy as np
nltk.download('stopwords')
from nltk.corpus import stopwords
import spacy
import spacy.cli
spacy.cli.download("pt_core_news_sm")
import pt_core_news_sm
spc_pt = pt_core_news_sm.load()
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split

#### Task 0: Carregar o dataset B2W-Reviews01-Anotado

In [ ]:
#Carrega o dataset anotado
data = pd.read_csv('../Documents/TCC/b2w-reviews-01-anotado.csv')

#### Task 1: Eliminar Reviews Neutras

In [ ]:
data = data[data['anotacao'] != 'neutro']
data.reset_index(drop=True, inplace=True)

In [ ]:
data.anotacao.value_counts()

#### Task 2: Preparação dos Dados para Modelos

In [ ]:
#Remove da lista de stopwords palavras importantes
stopwords_pt = stopwords.words("portuguese")
stopwords_pt = [i for i in stopwords_pt if i not in ["não", "nenhum", "nada", "jamais", "nunca", "nem"]]


def limpa_texto(texto):
    texto = texto.lower() #Transforma texto em minúsculo
    texto = re.sub(r"[\W\d_]+", " ", texto) #Filtra apenas letras
    texto = [pal for pal in texto.split() if pal not in stopwords_pt] #Remove stopwords
    spc_texto = spc_pt(" ".join(texto))
    tokens = [word.lemma_ if word.lemma_ != "-PRON-" else word.lower_ for word in spc_texto] #Lemmatiza

    return " ".join(tokens)

#Aplica função na coluna Review
data['review'] = data['review'].apply(limpa_texto)


#### Task 3: Feature extraction

Com Bag of Words

In [ ]:
# Instancia o CountVectorizer
vectorizer = CountVectorizer(binary=True)

# Vetoriza a coluna Review
texto = data['review']
X_bow = vectorizer.fit_transform(texto)

#Transforma o resultado em Array
X_bow.toarray()

Com TF-IDF

In [ ]:
# Instancia o TfidfVectorizer
tfidf_vect = TfidfVectorizer(max_features=5000)

# Vetoriza o texto
X_tfidf = tfidf_vect.fit_transform(texto)

print(X_tfidf)

#### Task 4: Binarizar as Classes de Sentimento

In [ ]:
#Lista as classes de sentimento existentes no dataset
SENTIMENTOS = ['positivo','negativo']

#Transforma as classes em coluna com valores preenchidos 
for sentimento in SENTIMENTOS[::-1]:
    data.insert(1,sentimento,data.anotacao.apply(lambda x: 1 if sentimento in x else 0))

#Visualização
data.head()

In [ ]:
# Converte os valores para Float
data['positivo'] = data['positivo'].astype(float)
data['negativo'] = data['negativo'].astype(float)
data.dtypes

##### Task 5: Dividir o Conjunto de Dados em Treino, Teste e Validação

Divisão entre Treino e Teste

In [ ]:
data.head()

In [ ]:
#Define as Variáveis Target e Predictor - BOW
X1 = X_bow
y = data[['positivo', 'negativo']]

In [ ]:
# Divide os dados em treino e teste - BOW
X_train,X_test,y_train,y_test = train_test_split(X1,y,test_size=0.2,random_state=42)

In [ ]:
#Define as Variáveis Target e Predictor - TD-IDF
X2 = X_tfidf
y = data[['positivo', 'negativo']]

In [ ]:
# Divide os dados em treino e teste - TD-IDF
X_train,X_test,y_train,y_test = train_test_split(X2,y,test_size=0.2,random_state=42)